In [1]:
!conda install -c conda-forge beautifulsoup4 --yes
!conda install -c conda-forge lxml --yes
!conda install -c conda-forge html5lib --yes
!conda install -c conda-forge requests --yes

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following packages will be UPDATED:

    beautifulsoup4: 4.6.0-py35h442a8c9_1 --> 4.6.3-py35_0 conda-forge

beautifulsoup4 100% |################################| Time: 0:00:00  41.61 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following packages will be UPDATED:

    libxml2: 2.9.4-h6b072ca_5     --> 2.9.8-h422b904_2     conda-forge
    libxslt: 1.1.29-hcf9102b_5    --> 1.1.32-h88dbc4e_2    conda-forge
    lxml:    4.1.0-py35ha401a81_0 --> 4.2.5-py35hc9114bc_0 conda-forge

libxml2-2.9.8- 100% |################################| Time: 0:00:00  63.00 MB/s
libxslt-1.1.32 100% |################################| Time: 0:00:00  68.82 MB/s
lxml-4.2.5-py3 100% |################################| Time: 0:00:00  60.15 MB/s
Fetchin

## 1. Explore the data

In [17]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)
data = response.text

soup = BeautifulSoup(data, 'lxml')

match = soup.table
table = match.contents[1]

x = 0
data = {}
data[0] = []
data[1] = []
data[2] = []

for child in table.contents:
    x = x + 1    
    if( x%2 != 0 ):
        data[0].append(child.get_text().strip().split('\n')[0])
        data[1].append(child.get_text().strip().split('\n')[1])
        data[2].append(child.get_text().strip().split('\n')[2])

df = pd.DataFrame({data[0][0]:data[0][1:],data[1][0]:data[1][1:],data[2][0]:data[2][1:]})
df.head()

,Borough,Neighbourhood,Postcode
0,Not assigned,Not assigned,M1A
1,Not assigned,Not assigned,M2A
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,Harbourfront,M5A


In [18]:
# Replace 'Not assigned' values from 'Borough' column with NaN values
df['Borough'].replace('Not assigned', np.nan, inplace=True)
# Delete rows with NaN values
df.dropna(subset=['Borough'], inplace=True)
df.head()

,Borough,Neighbourhood,Postcode
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,Harbourfront,M5A
5,Downtown Toronto,Regent Park,M5A
6,North York,Lawrence Heights,M6A


In [19]:
# Group files and separate with commas
df_group = df.groupby(['Postcode','Borough'], sort=False).agg( ','.join)
df_group.head()

,,Neighbourhood
Postcode,Borough,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront,Regent Park"
M6A,North York,"Lawrence Heights,Lawrence Manor"
M7A,Queen's Park,Not assigned


In [20]:
# Remove indexes
df_group.reset_index(drop=False, inplace=True)
# Asignm the same 'Borough' when 'Neighbourhood' is empty
df_group['Neighbourhood'].replace('Not assigned', df_group['Borough'], inplace=True)
df_group.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [21]:
df_group.shape

(103, 3)